# Group 22 Project Final Report

*An Analysis of the Risk of Developing Heart Disease for Individuals in Cleveland, Ohio.*

## Introduction

Cardiovascular disease includes any condition that affects the circulatory system. This results in negative impacts on the function of the heart and veins, which differ from different types of cardiovascular disease (Thiriet, 1970). Cardiovascular disease is also commonly known as heart disease (Thiriet, 1970). 
In order to save many lives, it is essential that individuals with heart disease are diagnosed early to start treatment as soon as possible (Pal et al., 2022). A model that can accurately predict the risk of developing cardiovascular disease would have the potential to save countless lives (Pal et al., 2022). 
By narrowing it down to specific factors such as cholesterol or blood pressure we can possibly give advice on proactive measures that are most important to minimize risk of heart disease.

Therefore the question we intend to answer is: Can we predict which patients in Cleveland, Ohio are at the highest risk of developing heart disease?

The dataset we will be using was retrieved from "processed.cleveland.data" in the Heart Disease Data Set (https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/) provided by UCI Machine Learning Repository. The dataset includes the following 14 attributes which can be numerical or categorical.

## Summary of Preliminary Analysis

We will start with a quick summary of our previous analysis. Our table describes which variables have apossible correlation with heart disease. We noticed some minor mistakes in our analysis but we will proceed with 2 numerical variables than seem to have correlation with heart disease

### Loading, Wrangling, and Cleaning the Data

In [1]:
# This cleaning-up procedure is mostly the same as our proposal
# We added some steps to omit rows containing unclassified values
import altair as alt
import numpy as np
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

np.random.seed(2023)

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"

original_data = pd.read_csv(url, names=[
    "age", 
    "sex", 
    "chest_pain", 
    "resting_blood_pressure", 
    "cholesterol", 
    "fasting_blood_sugar", 
    "resting_electrocardiographic_results", 
    "max_heart_rate",
    "exercise_induced_angina",  
    "st_depression_exercise", 
    "slope_st", 
    "major_vessels", 
    "thal",
    "diagnosis"
])

hd_data = original_data.dropna(how='all')

# omit rows containing unclassified values
hd_data = hd_data.drop(
    hd_data[hd_data.major_vessels == '?'].index
)
hd_data["major_vessels"] = pd.to_numeric(hd_data["major_vessels"])

hd_data = hd_data.drop(
    hd_data[hd_data.thal == '?'].index
)

In [3]:
# replace numeric values with categorical lables for categorical variables 
# except for 'resting_electrocardiographic_results' which requires a long description for each label
hd_data["sex"] = hd_data["sex"].apply(lambda x: "male" if (x == 1.0) else "female")

hd_data["chest_pain"] = hd_data["chest_pain"].replace({
    1: 'Typical angina',
    2: 'Atypical angina',
    3: 'Non-anginal pain',
    4: 'Asymptomatic'
})

hd_data["fasting_blood_sugar"] = hd_data["fasting_blood_sugar"].apply(lambda x: True if (x == 1.0) else False)

hd_data["exercise_induced_angina"] = hd_data["exercise_induced_angina"].apply(lambda x: 'Yes' if (x == 1.0) else 'No')

hd_data["slope_st"] = hd_data["slope_st"].replace({
    1.0: 'Upsloping',
    2.0: 'Flat',
    3.0: 'Downsloping'
})

hd_data["thal"] = hd_data["thal"].replace({
    '3.0': 'Normal',
    '6.0': 'Fixed defect',
    '7.0': 'Reversable defect'
})


In [4]:
# define a column, 'heart_disease', based on the 'diagnosis' column and drop 'diagnosis'
hd_data["heart_disease"] = hd_data["diagnosis"].apply(
    lambda x: "undiagnosed" if (x == 0) else "diagnosed")
hd_data = hd_data.drop(columns=["diagnosis"])

hd_data.head()

,age,sex,chest_pain,resting_blood_pressure,cholesterol,fasting_blood_sugar,resting_electrocardiographic_results,max_heart_rate,exercise_induced_angina,st_depression_exercise,slope_st,major_vessels,thal,heart_disease
0,63.0,male,Typical angina,145.0,233.0,True,2.0,150.0,No,2.3,Downsloping,0.0,Fixed defect,undiagnosed
1,67.0,male,Asymptomatic,160.0,286.0,False,2.0,108.0,Yes,1.5,Flat,3.0,Normal,diagnosed
2,67.0,male,Asymptomatic,120.0,229.0,False,2.0,129.0,Yes,2.6,Flat,2.0,Reversable defect,diagnosed
3,37.0,male,Non-anginal pain,130.0,250.0,False,0.0,187.0,No,3.5,Downsloping,0.0,Normal,undiagnosed
4,41.0,female,Atypical angina,130.0,204.0,False,2.0,172.0,No,1.4,Upsloping,0.0,Normal,undiagnosed


Now we have a 14-column dataset that contains six numerical variables and eight categorical variables. 

| # | Variable | Description | Value | Possible Correlation |
|---|----------|-------------|-------|----------------------|
| 1 | age | Individual's age | numerical | Yes; will check without the factor of sex |
| 2 | sex | Individual's sex | male or female | Yes |
| 3 | chest_pain | Chest pain type | Typical angina, Atypical angina, Non-anginal pain, or Asymptomatic | Yes |
| 4 | resting_blood_pressure | Resting blood pressure (in mm Hg on admission to the hospital) | numerical | worth checking a pattern |
| 5 | cholesterol | Serum cholestoral in mg/dL | numerical | re-analysis needed; will check distribution |
| 6 | fasting_blood_sugar | Fasting blood sugar > 120 mg/dL | True or False | No |
| 7 | resting_electrocardiographic_results | Resting electrocardiographic results | <ul><li>0: normal</li><li>1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)</li><li>2: showing probable or definite left ventricular hypertrophy by Estes' criteria</li></ul> | No |
| 8 | max_heart_rate | Maximum heart rate achieved | numerical | Yes; will check distribution |
| 9 | exercise_induced_angina | Exercise induced angina | Yes or No | Yes |
| 10 | st_depression_exercise | ST depression induced by exercise relative to rest | numerical | Yes; will check distribution |
| 11 | slope_st | The slope of the peak exercise ST segment | Upsloping, Flat, or Downsloping | Yes |
| 12 | major_vessels | Number of major vessels (0-3) colored by flourosopy | numerical | Yes |
| 13 | thal | Thalassemia | Normal, Fixed defect, or Reversable defect | Yes |
| 14 | heart_disease | Diagnosis of presence of heart disease | diagnosed or undiagnosed | NA |

### Exploratory Data Analysis

We will start to do some exploratory data analysis. First, we will split the dataset to do the exploratroy analysis on just the training data.

In [5]:
hd_train, hd_test = train_test_split(hd_data, test_size=0.25, random_state=123)

We will check some statistics for all six numeric variables in our training data, `hd_train`.

In [6]:
age_stat = hd_train.agg({'age': ['mean', 'median', 'min', 'max']}).round(decimals=1)
rbp_stat = hd_train.agg({'resting_blood_pressure': ['mean', 'median', 'min', 'max']}).round(decimals=2)
chol_stat = hd_train.agg({'cholesterol': ['mean', 'median', 'min', 'max']}).round(decimals=2)
max_hr_stat = hd_train.agg({'max_heart_rate': ['mean', 'median', 'min', 'max']}).round(decimals=2)
st_depression_exercise = hd_train.agg({'st_depression_exercise': ['mean', 'median', 'min', 'max']}).round(decimals=2)
major_vessels = hd_train.agg({'major_vessels': ['mean', 'median', 'min', 'max']}).round(decimals=2)

num_stat = pd.concat([age_stat, rbp_stat, chol_stat, max_hr_stat, st_depression_exercise, major_vessels], axis=1)
num_stat.transpose()

,mean,median,min,max
age,54.70,56.0,29.0,77.0
resting_blood_pressure,132.49,130.0,94.0,192.0
cholesterol,246.02,243.0,126.0,417.0
max_heart_rate,149.76,154.0,71.0,202.0
st_depression_exercise,1.02,0.6,0.0,5.6
major_vessels,0.68,0.0,0.0,3.0


We see some variations of scales in the above variables, so we will need to scale them for the later data analysis. However, we won't do so in our exploratory analysis because we will just compare the diagnosis variable, which is categorical, and one other variable.

Now, we will make some plots to see if there is a possible correlational relationship between a variable and the diagnosis. Note: the count of individuals in the following plots will only refer to the observations in the training data.

In [7]:
# define a custom theme to make plots readable (you can overwrite these settings by local config)
def readable_labels():
    return {
        'config': {
            'view': {'height': 300, 'width': 300},
            'title': {'fontSize': 18},
            "axisX": {"labelFontSize": 15, "titleFontSize": 15},
            "axisY": {"labelFontSize": 15, "titleFontSize": 15},
            "legend": {"labelFontSize": 15, "titleFontSize": 15},
        }
    }
alt.themes.register('readable_labels', readable_labels)
alt.themes.enable('readable_labels')

ThemeRegistry.enable('readable_labels')

In [8]:
age_hist = (alt.Chart()
            .mark_bar()
            .encode(
                x=alt.X("age", title="Age", bin=alt.Bin(maxbins=30)),
                y=alt.Y("count()", title="Count of individuals", stack=False),
            )
            .properties(height=150, width=400)
           )
age_hist_facet = (age_hist
                  .facet(
                      facet=alt.Facet("heart_disease", title="Heart disease", header=alt.Header(labelFontSize=15, titleFontSize=15)),
                      data=hd_train,
                      columns=1,
                      title='Age vs Heart Disease'
                  )
                 )
age_hist_facet

/opt/conda/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.FacetChart(...)

We can see the individuals who were diagnosed to heart disease appear within the range of 50s to 60s. On the other hand, we can see that individuals who were not diagnosed spread over the broad range from 30s to 70s.

In [9]:
sex_bar_graph = (alt.Chart(hd_train, title = "The Differentiation in the Number of Individuals with Heart Disease by Sex")
                 .mark_bar()
                 .encode(
                     # x=alt.X("count()", title= "Count of individuals"), 
                     x=alt.X("count()", title= "Count of individuals", stack="normalize", axis=alt.Axis(format='%')),
                     y=alt.Y("sex"),
                     color="heart_disease"
                 ).properties(width=380, height=150)
)
sex_bar_graph

alt.Chart(...)

Since we have fewer observations of females in our dataset, we compared the ratio of dianosis. By looking at the proportion of diagnosed individuals, we can see males show a larger proportion of being diagnosed for heart hisease than females.

In [10]:
chest_pain_graph = (
    alt.Chart(hd_train, title = "Chest Pain vs Heart Disease")
    .mark_bar()
    .encode(
        x=alt.X("count()", title="Count of individuals"), 
        y=alt.Y("chest_pain:N", title="Type of chest pain"),
        color="heart_disease"
    ).properties(width=380, height=200)
)
chest_pain_graph

alt.Chart(...)

Here we see that for the asymptomatic type of chest pain, the majority of people have been disagnosed with heart disease. Non-anginal pain is the second most recorded chest pain type, but the large portion of it is subjected to undiagnosed for heart disease.

When compared to the other types of chest pain, the data may indicate a slight correlation between an asymptomatic type of pain and heart disease; although, chest pain alone might not be a good indicator of heart disease.

In [11]:
resting_blood_pressure_graph = (
    alt.Chart(hd_train, title = "Resting Blood Pressure vs Heart Disease")  
    .mark_point()
    .encode(
        x=alt.X("resting_blood_pressure", title= "Resting Blood Pressure (mm Hg)", scale=alt.Scale(zero=False)), 
        y=alt.Y("count()", title= "Count of Individuals"),
        color="heart_disease"
    )
)
resting_blood_pressure_graph

alt.Chart(...)

There seems to be a weak pattern between resting blood pressure and the dianosis to heart disease;  individuals diagnosed to heart disease tend to appear in the right side of the plot, which area represents higher blood pressure.

In [12]:
cholesterol_hist = (alt.Chart()
            .mark_bar()
            .encode(
                x=alt.X("cholesterol", title="Serum cholesterol (mg/dL)", bin=alt.Bin(maxbins=30)),
                y=alt.Y("count()", title="Count of individuals"),
            )
            .properties(height=200, width=400)
           )
cholesterol_hist_facet = (cholesterol_hist
                  .facet(
                      facet=alt.Facet("heart_disease", title="Dianosis to Heart Disease", header=alt.Header(labelFontSize=15, titleFontSize=15)),
                      data=hd_train,
                      columns=1,
                      title="Cholesterol vs Heart Disease"
                  )
                 )
cholesterol_hist_facet

alt.FacetChart(...)

The distribution of the values of serum cholesterol (mg/dL) for diagnosed indivduals and that for undiagnosed individuals are very similar. It seems to be no pattern between heart disease and cholesterol.

In [13]:
# replace the values of fasting_blood_sugar with labels
hd_train["fasting_blood_sugar"] = hd_train["fasting_blood_sugar"].replace({
    True: '> 120 mg/dL',
    False: '<= 120 mg/dL'
})

fasting_blood_sugar_graph = (
    alt.Chart(hd_train, title = "Fasting blood sugar vs Heart Disease")  
    .mark_bar()
    .encode(
        x=alt.X("count()", title= "Count of Individuals"), color="heart_disease",
        y=alt.Y("fasting_blood_sugar", title= "Fasting blood sugar"), 
    ).properties(width=400, height=200)
)
fasting_blood_sugar_graph 

alt.Chart(...)

The proportions of diagnosed and undiagnosed indivisuals seem similar in both categories, so we will regard this as little to no correlation for now.

In [14]:
# replace the values of resting_electrocardiographic_results
hd_train["resting_electrocardiographic_results"] = hd_train[
    "resting_electrocardiographic_results"].replace({
    0: 'Normal',
    1: 'ST-T abnormality',
    2: 'Probable or definite left ventricular hypertrophy',
})

restecg_graph = (
    alt.Chart(hd_train, title = "Resting electrocardiographic results vs Heart Disease")  
    .mark_bar()
    .encode(
        x=alt.X("count()", title= "Count of Individuals"), color="heart_disease",
        y=alt.Y("resting_electrocardiographic_results:N", title=["Resting electrocardiographic", "results"]), 
    ).properties(width=400, height=150)
)
restecg_graph

alt.Chart(...)

There seems to be more individuals with the result of **probable or definite left ventricular hypertrophy by Estes' criteria** that are diagnosed with heart disease than for a result of **having ST-T wave abnormality**. We will consider this a small correlation for now. 

In [15]:
max_heart_rate_hist = (
    alt.Chart()
    .mark_bar()
    .encode(
        x=alt.X("max_heart_rate", title="Maximum heart rate achieved", bin=alt.Bin(maxbins=30)),
        y=alt.Y("count()", title="Count of individuals"),
    )
    .properties(height=150, width=400)
)
max_heart_rate_hist_facet = (
    max_heart_rate_hist
    .facet(
        facet=alt.Facet("heart_disease", title="Dianosis to Heart Disease", header=alt.Header(labelFontSize=15, titleFontSize=15)),
        data=hd_train,
        columns=1,
        title="Maximum Heart Rate Achieved vs Heart Disease"
    )
)
max_heart_rate_hist_facet

alt.FacetChart(...)

We can definitely see the distribution of maximu heart rate for undiagnosed individuals is right-skewed. We are not quite sure what this means but will keep this possible correlation in mind for future reference and decision on variables.

In [16]:
exercise_induced_angina_graph = (
    alt.Chart(hd_train, title = "Exercise Induced Angina vs Heart Disease")  
    .mark_bar()
    .encode(
        x=alt.X("count()", title= "Count of Individuals"),
        y=alt.Y("exercise_induced_angina", title=["Experience of", "exercise induced angina"]), 
        color="heart_disease"
    ).properties(width=300, height=150)
)
exercise_induced_angina_graph

alt.Chart(...)

The proportion of diagnosed individuals who experienced the exercise induced angina is higher than the proportion of undiagnosed people, which leads us to say there may be a correlation between this variable and heart disease.

In [17]:
st_depression_exercise_hist = (alt.Chart()
            .mark_bar()
            .encode(
                x=alt.X("st_depression_exercise", title="ST depression induced by exercise relative to rest", bin=alt.Bin(maxbins=30)),
                y=alt.Y("count()", title="Count of individuals"),
            )
            .properties(height=150, width=400)
           )
st_depression_exercise_hist_facet = (st_depression_exercise_hist
                  .facet(
                      facet=alt.Facet("heart_disease", title="Dianosis to Heart Disease", header=alt.Header(labelFontSize=15, titleFontSize=15)),
                      data=hd_train,
                      columns=1,
                      title="ST Depression Induced by Exercise vs Heart Disease"
                  )
                 )
st_depression_exercise_hist_facet

alt.FacetChart(...)

There seems to be a coreelation between heart disease diagnosis and ST depression induced by exercise relative to rest. Undiagnosed individuals are seen around 0 to 2.0, whereas diagnosed individuals are seen within a broader range at the higher ST depression rates.

In [18]:
slope_st_graph = (
    alt.Chart(hd_train, title=["The Slope of the Peak Exercise", "ST Segment vs Heart Disease"])  
    .mark_bar()
    .encode(
        x=alt.X("count()", title="Count of individuals"), color="heart_disease",
        y=alt.Y("slope_st", title=["Slope of the peak", "exercise ST segment"]),
    ).properties(width=400, height=150)
)
slope_st_graph

alt.Chart(...)

The proportion of individuals diagnosed with heart disease is highest for the flat/no slope. However, as we do not have much domain knowledge about reading the ST segment and have not deeply understood this variable from our searches, we will leave this variable out of our analysis.

In [19]:
major_vessels_graph = (
    alt.Chart(hd_train, title = "Number of Major Vessels vs Heart Disease")  
    .mark_bar()
    .encode(
        x=alt.X("major_vessels:N", title= "Number of major vessels (0-3) colored by fluoroscopy"), 
        y=alt.Y("count()", title= "Count of Individuals"), color="heart_disease"
    )
)
major_vessels_graph

alt.Chart(...)

The more major vessels are colored the higher the percentage of diagnosed individuals becomes, We think this could be an important variable for our question.

In [20]:
thalassemia_graph = (
    alt.Chart(hd_train, title = "Thalassemia vs Heart Disease")  
    .mark_bar()
    .encode(
        x=alt.X("count()", title= "Count of Individuals"), color="heart_disease",
        y=alt.Y("thal", title= "Presence of Thalassemia")
    ).properties(width=400, height=150)
)

thalassemia_graph

alt.Chart(...)

We can clearly see that there is a strong correlation between a diagnosis of heart disease and thalassemia from the large porition of diagnosed individuals in the category of "Reversable defect."

## Data Analysis with Selected Variables

### Scaling and Centering

We saw the variation of scales in the numeric variables in our data. So, we will scale these values before using them for our analysis with the K nearest neighbours algorithms.

In [21]:
hd_preprocessor = make_column_transformer(
    (StandardScaler(), [
        "age",
        "resting_blood_pressure",
        "cholesterol",
        "max_heart_rate",
        "st_depression_exercise",
        "major_vessels"
    ]),
    ("passthrough", [
        "sex",
        "chest_pain",
        "fasting_blood_sugar",
        "resting_electrocardiographic_results",
        "exercise_induced_angina",
        "slope_st", 
        "thal",
        "heart_disease"
    ]),
)

hd_scaled = pd.DataFrame(
    hd_preprocessor.fit_transform(hd_data),
    columns=[
        "age",
        "resting_blood_pressure",
        "cholesterol",
        "max_heart_rate",
        "st_depression_exercise",
        "major_vessels",
        "sex",
        "chest_pain",
        "fasting_blood_sugar",
        "resting_electrocardiographic_results",
        "exercise_induced_angina",
        "slope_st", 
        "thal",
        "heart_disease"
    ],
)
hd_scaled.head()

,age,resting_blood_pressure,cholesterol,max_heart_rate,st_depression_exercise,major_vessels,sex,chest_pain,fasting_blood_sugar,resting_electrocardiographic_results,exercise_induced_angina,slope_st,thal,heart_disease
0,0.936181,0.75038,-0.276443,0.017494,1.068965,-0.721976,male,Typical angina,True,2.0,No,Downsloping,Fixed defect,undiagnosed
1,1.378929,1.596266,0.744555,-1.816334,0.381773,2.478425,male,Asymptomatic,False,2.0,Yes,Flat,Normal,diagnosed
2,1.378929,-0.659431,-0.3535,-0.89942,1.326662,1.411625,male,Asymptomatic,False,2.0,Yes,Flat,Reversable defect,diagnosed
3,-1.94168,-0.095506,0.051047,1.63301,2.099753,-0.721976,male,Non-anginal pain,False,0.0,No,Downsloping,Normal,undiagnosed
4,-1.498933,-0.095506,-0.835103,0.978071,0.295874,-0.721976,female,Atypical angina,False,2.0,No,Upsloping,Normal,undiagnosed


### Training Data and Test Data

We will obtain a training data and a test data from the `hd_scaled` data and use only *the scaled training data*, `hd_scaled_train`, from now on. We will stratify the target column `heart_disease` to preserve the class proportion.

In [22]:
from sklearn.model_selection import train_test_split

hd_scaled_train, hd_scaled_test = train_test_split(hd_scaled, test_size=0.25, stratify=hd_scaled["heart_disease"])
print(hd_scaled_train["heart_disease"].value_counts(normalize=True))
print(hd_scaled_test["heart_disease"].value_counts(normalize=True))

undiagnosed    0.540541
diagnosed      0.459459
Name: heart_disease, dtype: float64
undiagnosed    0.533333
diagnosed      0.466667
Name: heart_disease, dtype: float64
